In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
pdlist = ['DATE','DATETIME']
df = pd.read_csv('all_turnstiles_9-2016-9-2017.csv', parse_dates=pdlist)

In [3]:
df.columns

Index(['Unnamed: 0', 'STATION', 'C/A', 'UNIT', 'SCP', 'DATE', 'DATETIME',
       'D_ENTRIES', 'D_EXITS'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10563412 entries, 0 to 10563411
Data columns (total 9 columns):
Unnamed: 0    int64
STATION       object
C/A           object
UNIT          object
SCP           object
DATE          datetime64[ns]
DATETIME      datetime64[ns]
D_ENTRIES     float64
D_EXITS       float64
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 725.3+ MB


In [5]:
df['TOTAL_ENTRIES_EXITS'] = df['D_ENTRIES'] + df['D_EXITS']
df['WEEKDAY'] = df['DATE'].apply(lambda x: datetime.weekday(x))

In [6]:
def morn_aft_evening_late(input_datetime):
    if input_datetime.hour >= 5 and input_datetime.hour < 13:
        return 0
    elif input_datetime.hour >= 13 and input_datetime.hour < 18:
        return 1
    elif input_datetime.hour >= 18 and input_datetime.hour < 23:
        return 2
    else:
        return 3

In [7]:
df['MORN_AFT_EVENING_LATE'] = df['DATETIME'].apply(morn_aft_evening_late)
df['HOUR'] = df['DATETIME'].apply(lambda x: x.hour)

In [11]:
dfout = df.groupby(['STATION','WEEKDAY','MORN_AFT_EVENING_LATE'], as_index=False).sum()

In [14]:
def get_ratio(row):
    if row["TOTAL_ENTRIES_EXITS"] == 0:
        return 0
    ratio = 2 * (row["D_ENTRIES"] / row["TOTAL_ENTRIES_EXITS"]) - 1
    return ratio

In [15]:
dfout['RATIO'] = dfout.apply(get_ratio, axis=1)

In [17]:
dfout

,STATION,WEEKDAY,MORN_AFT_EVENING_LATE,Unnamed: 0,D_ENTRIES,D_EXITS,TOTAL_ENTRIES_EXITS,HOUR,RATIO
0,1 AV,0,0,12564995,305402.0,402730.0,708132.0,10626,-0.137443
1,1 AV,0,1,6161642,208240.0,195481.0,403721.0,8460,0.031604
2,1 AV,0,2,6251844,336675.0,314600.0,651275.0,10720,0.033895
3,1 AV,0,3,12320542,176853.0,186448.0,363301.0,6120,-0.026411
4,1 AV,1,0,12722688,337354.0,446212.0,783566.0,10780,-0.138926
5,1 AV,1,1,6164922,215278.0,200343.0,415621.0,8460,0.035934
6,1 AV,1,2,6165462,362027.0,331117.0,693144.0,10620,0.044594
7,1 AV,1,3,12303527,213846.0,208069.0,421915.0,6100,0.013692
8,1 AV,2,0,12774662,350949.0,461281.0,812230.0,10710,-0.135838
9,1 AV,2,1,6168197,227089.0,209871.0,436960.0,8460,0.039404


In [18]:
dfout.to_csv('9-2016_to_9-2017_ratios.csv')